In [ ]:
# БАЗОВЫЕ ИМПОРТЫ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# МОДЕЛИ
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing
from sklearn.linear_model import LinearRegression

# МЕТРИКИ
from sklearn.metrics import mean_absolute_error, mean_squared_error

# --------- ВАРИАНТ A: читаем ваши данные ---------
# Ожидаемый формат: две колонки ['date', 'value'], где 'date' — дата/время, 'value' — числовой ряд
# df = pd.read_csv('your_timeseries.csv')  # или pd.read_excel('file.xlsx', sheet_name='...')

# --------- ВАРИАНТ B: создадим игрушечный пример на месяцах ---------
rng = pd.date_range('2021-01-01', periods=48, freq='MS')  # 48 месяцев, начало месяца
np.random.seed(42)
# тренд + лёгкая сезонность + шум
vals = 100 + np.arange(len(rng))*0.8 + 5*np.sin(2*np.pi*rng.month/12) + np.random.normal(0, 1.5, len(rng))
df = pd.DataFrame({'date': rng, 'value': vals})

# ----- ОБЩАЯ ПОДГОТОВКА -----
# 1) приведение к datetime
df['date'] = pd.to_datetime(df['date'], errors='coerce')
# 2) сортировка и установка индекса
df = df.sort_values('date').set_index('date')
# 3) явная частота (например, месячная)
df = df.asfreq('MS')   # 'MS' = Month Start; подставьте свою: 'D', 'W', 'QS', 'H' и т.п.
# 4) обработка пропусков (если есть)
df['value'] = df['value'].interpolate()  # или forward-fill: .ffill()

# Разделим на train/test: последние 12 периодов оставим на тест
h = 12
train = df.iloc[:-h].copy()
test  = df.iloc[-h:].copy()

print(train.shape, test.shape)

'''
2) Метод №1 — Экспоненциальное сглаживание
Как работает (механика, коротко)

Экспоненциальное сглаживание обновляет оценку уровня ряда рекурсивно, сильнее “веря” свежим наблюдениям:
Sᵗ = α·Yᵗ + (1−α)·Sᵗ⁻¹, где

Yᵗ — фактическое значение, Sᵗ — сглаженный уровень,

α (alpha) ∈ (0,1) — вес последних точек (чем выше, тем быстрее модель реагирует на изменения).

Расширения:

Holt: добавляет тренд (уровень + тренд).

Holt-Winters: добавляет ещё и сезонность (аддитивную/мультипликативную).

2.1 Simple Exponential Smoothing (без тренда и сезонности)
'''

In [ ]:
# --- Simple Exponential Smoothing (SES) ---
ses = SimpleExpSmoothing(train['value'], initialization_method='estimated')
ses_fit = ses.fit(optimized=True)  # подберёт alpha и начальные значения
ses_forecast = ses_fit.forecast(steps=h)

print("SES params:", ses_fit.params)  # в т.ч. alpha


# --- Holt (линейный тренд, без сезонности) ---
holt = ExponentialSmoothing(
    train['value'],
    trend='add',        # 'add' или 'mul' — аддитивный/мультипликативный тренд
    seasonal=None,      # без сезонности
    initialization_method='estimated'
)
holt_fit = holt.fit(optimized=True)
holt_forecast = holt_fit.forecast(steps=h)

print("Holt params:", holt_fit.params)  # alpha, beta, etc.

'''
(Опционально) Holt-Winters (если у вас явная сезонность)

Если у вас месячные данные и выраженная годовая сезонность — добавьте seasonal='add'/'mul' и seasonal_periods=12.
'''

# --- Holt-Winters (уровень + тренд + сезонность) ---
hw = ExponentialSmoothing(
    train['value'],
    trend='add',
    seasonal='add',      # или 'mul' при относительной сезонности
    seasonal_periods=12, # длина сезона
    initialization_method='estimated'
)
hw_fit = hw.fit(optimized=True)
hw_forecast = hw_fit.forecast(steps=h)

print("Holt-Winters params:", hw_fit.params)


def mape(y_true, y_pred):
    # защита от деления на ноль
    denom = np.where(np.abs(y_true) < 1e-12, 1e-12, np.abs(y_true))
    return np.mean(np.abs((y_true - y_pred) / denom)) * 100

def evaluate_and_plot(model_name, fit_obj, forecast, color_actual='black'):
    mae = mean_absolute_error(test['value'], forecast)
    rmse = mean_squared_error(test['value'], forecast, squared=False)
    mp = mape(test['value'].values, forecast.values)
    print(f"{model_name} -> MAE={mae:.3f}, RMSE={rmse:.3f}, MAPE={mp:.2f}%")

    plt.figure(figsize=(10,4))
    plt.plot(train.index, train['value'], label='train')
    plt.plot(test.index,  test['value'],  label='test', linewidth=2)
    plt.plot(test.index,  forecast,       label=f'{model_name} forecast', linestyle='--')
    plt.title(f'{model_name}: прогноз на {h} шагов')
    plt.legend()
    plt.tight_layout()
    plt.show()

# Оценим SES
evaluate_and_plot("SES", ses_fit, ses_forecast)

# Оценим Holt
evaluate_and_plot("Holt", holt_fit, holt_forecast)

# (Опционально) Holt-Winters
# evaluate_and_plot("Holt-Winters", hw_fit, hw_forecast)

'''
Когда применять:

SES — когда нет явного тренда/сезонности.

Holt — когда есть тренд, но нет сезонности.

Holt-Winters — когда есть тренд и сезонность.
Модели обучаются быстро, мало гиперпараметров, хорошо “держат” плавные ряды.

ОБЩИЙ СИНТАКСИС
'''
model = ExponentialSmoothing(
    train['value'],
    trend='add',              # тип тренда ('add' или 'mul')
    seasonal='add',           # тип сезонности ('add' или 'mul', или None)
    seasonal_periods=12,      # длина сезона, если seasonal не None
    initialization_method='estimated'
)

fit = model.fit(optimized=True)
forecast = fit.forecast(steps=h)
